<h1 style=color:#4389db>Visualize The Word Embeddings Generated</h1>

<h2 style=color:#4389db>Download the Dataset</h2>

In [1]:
import tensorflow_datasets as tfds

# Load the IMDB Reviews dataset
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

C:\Users\Hussein\mypython\myvenv\Data101\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h2 style=color:#4389db>Split the dataset</h2>

In [7]:
import numpy as np

# Get the train and test sets
train_data, test_data = imdb['train'], imdb['test']

# Initialize sentences and labels lists
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# Loop over all training examples and save the sentences and labels
for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

# Loop over all test examples and save the sentences and labels
for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

# Convert labels lists to numpy array
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

<h2 style=color:#4389db>Generate Padded Sequences</h2>

In [8]:
# Parameters

vocab_size = 10000
max_length = 120
embedding_dim = 16
trunc_type='post'
oov_tok = "<OOV>"

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer class
tokenizer = Tokenizer(
                      num_words = vocab_size,
                      oov_token=oov_tok
                      )

# Generate the word index dictionary for the training sentences
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(
                       sequences,
                       maxlen=max_length,
                       truncating=trunc_type
                       )


# Generate and pad the test sequences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(
                               testing_sequences,
                               maxlen=max_length
                               )

In [28]:
# word_index

<h2 style=color:#4389db>Build and Compile the Model</h2>

In [10]:
import tensorflow as tf

# Build the model
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(
                                                       vocab_size,
                                                       embedding_dim,
                                                       input_length=max_length
                                                       ),
    
                             tf.keras.layers.GlobalAveragePooling1D(),
    
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
                             ])

# Setup the training parameters
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 6)                 102       
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


<h2 style=color:#4389db>Train the Model</h2>

In [11]:
num_epochs = 10

# Train the model
model.fit(
          padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final)
          )

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5450 - accuracy: 0.7444 - val_loss: 0.3814 - val_accuracy: 0.8403
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.3293 - accuracy: 0.8634 - val_loss: 0.3282 - val_accuracy: 0.8598
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2685 - accuracy: 0.8912 - val_loss: 0.3289 - val_accuracy: 0.8585
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2324 - accuracy: 0.9102 - val_loss: 0.3459 - val_accuracy: 0.8542
Epoch 5/10
782/782 [==============================] - 4s 6ms/step - loss: 0.2066 - accuracy: 0.9228 - val_loss: 0.3643 - val_accuracy: 0.8481
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1854 - accuracy: 0.9330 - val_loss: 0.3915 - val_accuracy: 0.8412
Epoch 7/10
782/782 [==============================] - 4s 6ms/step - loss: 0.1688 - accuracy: 0.9399 - val_loss: 0.4215 - val_accuracy: 0.8366
Epoch 

<h2 style=color:#4389db>Visualize Word Embeddings</h2>

In [12]:
# Get the embedding layer from the model (i.e. first layer)
embedding_layer = model.layers[0]

# Get the weights of the embedding layer
embedding_weights = embedding_layer.get_weights()[0]

# Print the shape. Expected is (vocab_size, embedding_dim)
print(embedding_weights.shape)

(10000, 16)


In [23]:
# Get the index-word dictionary
reverse_word_index = tokenizer.index_word

In [27]:
# reverse_word_index

In [29]:
import io

# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# Initialize the loop. Start counting at `1` because `0` is just for the padding
for word_num in range(1, vocab_size):

  # Get the word associated at the current index
  word_name = reverse_word_index[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = embedding_weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

# Close the files
out_v.close()
out_m.close()